In [ ]:
from env.core import InstanceType, Param, And, Or, IntBoolean
from env.full_state import FullState
from env.meta_env import MetaInfo
from env.goal_env import GoalEnv
from env.node_types import HaveScratch

params = (Param.from_int(1), Param.from_int(2), Param.from_int(3))
p1, p2, p3 = params
goal = HaveScratch.with_goal(
    InstanceType(
        Or(
            And(p1, p2, IntBoolean(1)),
            And(p2, p3),
        ),
    )
)

env = GoalEnv(goal)
node_types = env.full_state.node_types()
print('Amount of node types:', len(node_types))

In [ ]:
selected_goal = env.full_state.nested_arg((FullState.idx_meta, MetaInfo.idx_goal)).apply()
assert selected_goal == goal

In [ ]:
env.symbol(selected_goal).symbol

In [ ]:
env.to_node_data(selected_goal).to_data_array()

In [ ]:
env.current_state_symbol.symbol

In [ ]:
env.to_node_data(env.full_state.current_state.apply()).to_data_array()

In [ ]:
len(env.to_data().to_data_array())

In [ ]:
env.full_state.validate()

In [ ]:
from env.full_state import HistoryNode
from env.state import State
def get_current_state(env: GoalEnv):
    return env.full_state.nested_arg((FullState.idx_current, HistoryNode.idx_state)).apply().cast(State)
current_state = get_current_state(env)
env.symbol(current_state).symbol

In [ ]:
from env.state import State
assert current_state == State.create()

In [ ]:
from env.action_impl import CreateScratch
action = CreateScratch.create()
env.step(action)
current_state = get_current_state(env)
env.symbol(current_state).symbol

In [ ]:
assert current_state == State.from_raw(
    scratches=tuple([None]),
    args_groups=tuple(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.full_state import HistoryGroupNode
from env.core import Optional
def get_last_history_action(env: GoalEnv):
    history = env.full_state.history.apply().cast(HistoryGroupNode)
    last = history.as_tuple[-1]
    return last.action_data.apply().inner_arg(Optional.idx_value).apply()
last_history_action = get_last_history_action(env)
env.symbol(last_history_action).symbol

In [ ]:
from env.action import SuccessActionData
from env.action_impl import CreateScratchOutput
from env.state import StateScratchIndex, Scratch

assert last_history_action == SuccessActionData.from_args(
    action=Optional(CreateScratch(Optional())),
    output=Optional(CreateScratchOutput(
        StateScratchIndex(1),
        Scratch(),
    )),
    exception=Optional(),
)

In [ ]:
from env.action_impl import DefineScratchFromInt
# should return an exception (index won't return element)
action = DefineScratchFromInt.from_raw(1, len(node_types)+1, 3)
env.step(action)
current_state = get_current_state(env)
env.symbol(current_state).symbol

In [ ]:
assert current_state == State.from_raw(
    scratches=tuple([None]),
    args_groups=tuple(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
last_history_action = get_last_history_action(env)
env.symbol(last_history_action).symbol

In [ ]:
from env.action import IAction, ActionErrorActionData
from env.core import BooleanExceptionInfo, IsEmpty

def get_empty_exception(action: IAction):
    return ActionErrorActionData.from_args(
        action=Optional(action),
        output=Optional(),
        exception=Optional(BooleanExceptionInfo(IsEmpty(Optional()))),
    )

assert last_history_action == get_empty_exception(action)

In [ ]:
from env.action_impl import DefineScratchFromIntIndex
from env.core import INode
from env.meta_env import SubtypeOuterGroup, GeneralTypeGroup
from env.full_state import MetaAllTypesTypeIndex

def get_info_type_index(node_type: type[INode], env: GoalEnv, node_types: tuple[type[INode], ...]):
    index_type_idx = node_types.index(MetaAllTypesTypeIndex) + 1
    type_node = node_types[index_type_idx-1].as_type()
    selected_types = env.full_state.meta.apply().nested_arg((
        MetaInfo.idx_full_state_int_index_group,
        SubtypeOuterGroup.idx_subtypes,
    )).apply().cast(GeneralTypeGroup)
    meta_idx = selected_types.as_tuple.index(type_node) + 1
    node_idx = node_types.index(node_type) + 1
    return meta_idx, node_idx

meta_idx, type_idx = get_info_type_index(And, env, node_types)
action = DefineScratchFromIntIndex.from_raw(1, meta_idx, type_idx)
env.step(action)
current_state = get_current_state(env)
env.symbol(current_state).symbol

In [ ]:
assert current_state == State.from_raw(
    scratches=tuple([And.as_type()]),
    args_groups=tuple(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
last_history_action = get_last_history_action(env)
env.symbol(last_history_action).symbol

In [ ]:
from env.core import Integer
from env.full_state import MetaFullStateIntIndexTypeIndex
from env.action_impl import DefineScratchOutput
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineScratchFromIntIndex(
        StateScratchIndex(1),
        MetaFullStateIntIndexTypeIndex(meta_idx),
        Integer(type_idx)
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(1),
        Scratch(And.as_type()),
    )),
    exception=Optional(),
)

In [ ]:
action = CreateScratch.create()
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
assert current_state == State.from_raw(
    scratches=tuple([And.as_type(), None]),
    args_groups=tuple(),
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(CreateScratch(Optional())),
    output=Optional(CreateScratchOutput(
        StateScratchIndex(2),
        Scratch(),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
meta_idx, type_idx = get_info_type_index(Or, env, node_types)
action = DefineScratchFromIntIndex.from_raw(2, meta_idx, type_idx)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
assert current_state == State.from_raw(
    scratches=tuple([And.as_type(), Or.as_type()]),
    args_groups=tuple(),
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineScratchFromIntIndex(
        StateScratchIndex(2),
        MetaFullStateIntIndexTypeIndex(meta_idx),
        Integer(type_idx)
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(2),
        Scratch(Or.as_type()),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
action = CreateScratch.create()
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
assert current_state == State.from_raw(
    scratches=tuple([And.as_type(), Or.as_type(), None]),
    args_groups=tuple(),
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(CreateScratch(Optional())),
    output=Optional(CreateScratchOutput(
        StateScratchIndex(3),
        Scratch(),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import CreateArgsGroup
action = CreateArgsGroup.from_raw(3, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)
env.symbol(current_state).symbol

In [ ]:
from env.state import PartialArgsGroup, StateArgsGroupIndex
from env.action_impl import CreateArgsGroupOutput

arg_group = PartialArgsGroup(
    Optional(),
    Optional(),
    Optional(),
)

assert current_state == State.from_raw(
    scratches=tuple([And.as_type(), Or.as_type(), None]),
    args_groups=tuple([arg_group]),
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(CreateArgsGroup(
        Integer(3),
        Optional(),
    )),
    output=Optional(CreateArgsGroupOutput(
        StateArgsGroupIndex(1),
        arg_group,
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineScratchFromInt

def get_from_int_type_index(node_type: type[INode], env: GoalEnv):
    selected_types = env.full_state.meta.apply().nested_arg((
        MetaInfo.idx_from_int_group,
        SubtypeOuterGroup.idx_subtypes,
    )).apply().cast(GeneralTypeGroup)
    meta_idx = selected_types.as_tuple.index(node_type.as_type()) + 1
    return meta_idx

meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 1)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
from env.full_state import MetaFromIntTypeIndex
assert current_state == State.from_raw(
    scratches=tuple([And.as_type(), Or.as_type(), Param.from_int(1)]),
    args_groups=tuple([arg_group]),
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(1),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Scratch(Param.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(1, 1, 3)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
from env.core import NodeArgIndex
from env.action_impl import DefineArgsGroupArgOutput

arg_group = PartialArgsGroup(
    Optional(Param.from_int(1)),
    Optional(),
    Optional(),
)

assert current_state == State.from_raw(
    scratches=tuple([And.as_type(), Or.as_type(), Param.from_int(1)]),
    args_groups=tuple([arg_group]),
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(1),
        NodeArgIndex(1),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(1),
        NodeArgIndex(1),
        Scratch(Param.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 2)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
from env.full_state import MetaFromIntTypeIndex
assert current_state == State.from_raw(
    scratches=tuple([And.as_type(), Or.as_type(), Param.from_int(2)]),
    args_groups=tuple([arg_group]),
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(2),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Scratch(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(1, 2, 3)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
from env.core import NodeArgIndex
from env.action_impl import DefineArgsGroupArgOutput

arg_group = PartialArgsGroup(
    Optional(Param.from_int(1)),
    Optional(Param.from_int(2)),
    Optional(),
)

assert current_state == State.from_raw(
    scratches=tuple([And.as_type(), Or.as_type(), Param.from_int(2)]),
    args_groups=tuple([arg_group]),
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(1),
        NodeArgIndex(2),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(1),
        NodeArgIndex(2),
        Scratch(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
meta_idx = get_from_int_type_index(IntBoolean, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 1)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
from env.full_state import MetaFromIntTypeIndex
assert current_state == State.from_raw(
    scratches=tuple([And.as_type(), Or.as_type(), IntBoolean.from_int(1)]),
    args_groups=tuple([arg_group]),
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(1),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Scratch(IntBoolean.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(1, 3, 3)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
from env.core import NodeArgIndex
from env.action_impl import DefineArgsGroupArgOutput

arg_group = PartialArgsGroup(
    Optional(Param.from_int(1)),
    Optional(Param.from_int(2)),
    Optional(IntBoolean.from_int(1)),
)

scratches = tuple([And.as_type(), Or.as_type(), IntBoolean.from_int(1)])
args_groups = tuple([arg_group])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(1),
        NodeArgIndex(3),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(1),
        NodeArgIndex(3),
        Scratch(IntBoolean.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import CreateArgsGroup
action = CreateArgsGroup.from_raw(2, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
arg_group_2 = PartialArgsGroup(
    Optional(),
    Optional(),
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(CreateArgsGroup(
        Integer(2),
        Optional(),
    )),
    output=Optional(CreateArgsGroupOutput(
        StateArgsGroupIndex(2),
        arg_group_2,
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 2)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
scratches = tuple([And.as_type(), Or.as_type(), Param.from_int(2)])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(2),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Scratch(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 1, 3)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
arg_group_2 = PartialArgsGroup(
    Optional(Param.from_int(2)),
    Optional(),
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        Scratch(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 3)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
scratches = tuple([And.as_type(), Or.as_type(), Param.from_int(3)])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(3),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Scratch(Param.from_int(3)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 2, 3)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
arg_group_2 = PartialArgsGroup(
    Optional(Param.from_int(2)),
    Optional(Param.from_int(3)),
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        Scratch(Param.from_int(3)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineScratchFromFunctionWithArgs
action = DefineScratchFromFunctionWithArgs.from_raw(3, 1, 1)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
call_1 = And(
    Param.from_int(1),
    Param.from_int(2),
    IntBoolean.from_int(1),
)
scratches = tuple([And.as_type(), Or.as_type(), call_1])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineScratchFromFunctionWithArgs(
        StateScratchIndex(3),
        StateScratchIndex(1),
        Optional(StateArgsGroupIndex(1)),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Scratch(call_1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineScratchFromFunctionWithArgs
action = DefineScratchFromFunctionWithArgs.from_raw(1, 1, 2)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
call_2 = And(
    Param.from_int(2),
    Param.from_int(3),
)
scratches = tuple([call_2, Or.as_type(), call_1])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineScratchFromFunctionWithArgs(
        StateScratchIndex(1),
        StateScratchIndex(1),
        Optional(StateArgsGroupIndex(2)),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(1),
        Scratch(call_2),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 1, 3)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
arg_group_2 = PartialArgsGroup(
    Optional(call_1),
    Optional(Param.from_int(3)),
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        Scratch(call_1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 2, 1)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
arg_group_2 = PartialArgsGroup(
    Optional(call_1),
    Optional(call_2),
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        StateScratchIndex(1),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        Scratch(call_2),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineScratchFromFunctionWithArgs
action = DefineScratchFromFunctionWithArgs.from_raw(1, 2, 2)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
call_3 = Or(
    call_1,
    call_2,
)
scratches = tuple([call_3, Or.as_type(), call_1])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DefineScratchFromFunctionWithArgs(
        StateScratchIndex(1),
        StateScratchIndex(2),
        Optional(StateArgsGroupIndex(2)),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(1),
        Scratch(call_3),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
env.symbol(current_state).symbol

In [ ]:
env.to_node_data(current_state).to_data_array()

In [ ]:
from env.action_impl import DeleteArgsGroupOutput
action = DeleteArgsGroupOutput.from_raw(1, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
args_groups = tuple([arg_group_2])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    output=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DeleteArgsGroupOutput
action = DeleteArgsGroupOutput.from_raw(1, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
args_groups = tuple()

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    output=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DeleteScratchOutput
action = DeleteScratchOutput.from_raw(3, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
scratches = tuple([call_3, Or.as_type()])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DeleteScratchOutput(
        StateScratchIndex(3),
    )),
    output=Optional(DeleteScratchOutput(
        StateScratchIndex(3),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DeleteScratchOutput
action = DeleteScratchOutput.from_raw(2, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
scratches = tuple([call_3])

assert current_state == State.from_raw(
    scratches=scratches,
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(DeleteScratchOutput(
        StateScratchIndex(2),
    )),
    output=Optional(DeleteScratchOutput(
        StateScratchIndex(2),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import DefineScratchFromSingleArg

def get_single_child_type_index(node_type: type[INode], env: GoalEnv):
    selected_types = env.full_state.meta.apply().nested_arg((
        MetaInfo.idx_single_child_group,
        SubtypeOuterGroup.idx_subtypes,
    )).apply().cast(GeneralTypeGroup)
    meta_idx = selected_types.as_tuple.index(node_type.as_type()) + 1
    return meta_idx

meta_idx = get_single_child_type_index(InstanceType, env)
action = DefineScratchFromSingleArg.from_raw(1, meta_idx, 1)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
env.symbol(current_state).symbol

In [ ]:
env.to_node_data(current_state).to_data_array()

In [ ]:
from env.state import Scratch
content = current_state.nested_arg((
    State.idx_scratch_group,
    1,
    Scratch.idx_value,
)).apply()
assert content == goal.goal_inner_expr.apply()

In [ ]:
from env.state import StateScratchIndex
verification = goal.evaluate(current_state, StateScratchIndex(1))
verification.raise_on_false()
verification.as_bool

In [ ]:
from env.action_impl import CreateScratch
action = CreateScratch.create()
env.step(action)

meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(2, meta_idx, 4)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
from env.state import StateScratchIndex
verification = goal.evaluate(current_state, StateScratchIndex(2))
verification.as_bool

In [ ]:
action = DeleteScratchOutput.from_raw(2, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
from env.action_impl import VerifyGoal
from env.state import StateScratchIndex
meta_idx = get_from_int_type_index(StateScratchIndex, env)
action = VerifyGoal.from_raw(0, meta_idx, 1)
env.step(action)
current_state = get_current_state(env)
last_history_action = get_last_history_action(env)

In [ ]:
env.symbol(current_state).symbol

In [ ]:
env.symbol(last_history_action).symbol

In [ ]:
from env.action_impl import VerifyGoalOutput
from env.state import StateMetaInfo, GoalAchieved

scratch_goal = InstanceType(call_3)

assert current_state == State.from_raw(
    meta_info=StateMetaInfo.with_goal_achieved(GoalAchieved.achieved()),
    scratches=[scratch_goal],
    args_groups=args_groups,
)
assert last_history_action == SuccessActionData.from_args(
    action=Optional(VerifyGoal(
        Optional(),
        MetaFromIntTypeIndex(meta_idx),
        Integer(1),
    )),
    output=Optional(VerifyGoalOutput(
        Optional(),
        StateScratchIndex(1),

    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is True